In [1]:
!pip install -q transformers datasets peft accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 

In [2]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install wandb

In [4]:
import wandb
!wandb login 4842d7a52a0d7c51c20fbd9783da089097f43bf3

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, default_data_collator
import torch

2025-05-02 12:57:30.715015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746190650.928497      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746190650.983189      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
dataset = load_dataset("squad_v2")

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [7]:
dataset = dataset.shuffle(seed=42)

In [8]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [9]:
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

In [10]:
train_data = split_dataset["train"]
test_data = split_dataset["test"]
val_data = dataset["validation"]

In [11]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Get the corresponding original example
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        # If no answer, set labels to CLS token
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
            continue

        # Get start/end character positions
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])

        # Find start/end token indices (context only)
        token_start_index = 0
        while token_start_index < len(sequence_ids) and sequence_ids[token_start_index] != 1:
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while token_end_index >= 0 and sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # Check if answer is out of span
        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
            continue

        # Find token positions
        start_position = token_start_index
        end_position = token_end_index
        for idx in range(token_start_index, token_end_index + 1):
            if offsets[idx][0] <= start_char:
                start_position = idx
            if offsets[idx][1] >= end_char:
                end_position = idx
                break

        tokenized_examples["start_positions"].append(start_position)
        tokenized_examples["end_positions"].append(end_position)

    return tokenized_examples



# Tokenize dataset
tokenized_train = train_data.map(
    prepare_train_features,
    batched=True,
    remove_columns=train_data.column_names
)

tokenized_test = test_data.map(
    prepare_train_features,
    batched=True,
    remove_columns=test_data.column_names
)

tokenized_val = val_data.map(
    prepare_train_features,
    batched=True,
    remove_columns=val_data.column_names
)

Map:   0%|          | 0/117287 [00:00<?, ? examples/s]

Map:   0%|          | 0/13032 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [13]:
# def prepare_train_features(examples):
#     tokenized_examples = tokenizer(
#         examples["question"],
#         examples["context"],
#         truncation="only_second",
#         max_length=384,
#         stride=64,
#         return_overflowing_tokens=True,
#         return_offsets_mapping=True,
#         padding="max_length"
#     )

#     sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
#     offset_mapping = tokenized_examples.pop("offset_mapping")

#     tokenized_examples["start_positions"] = []
#     tokenized_examples["end_positions"] = []

#     for i, offsets in enumerate(offset_mapping):
#         input_ids = tokenized_examples["input_ids"][i]
#         cls_index = input_ids.index(tokenizer.cls_token_id)

#         sequence_ids = tokenized_examples.sequence_ids(i)
#         sample_index = sample_mapping[i]
#         answers = examples["answers"][sample_index]

#         if len(answers["answer_start"]) == 0:
#             tokenized_examples["start_positions"].append(cls_index)
#             tokenized_examples["end_positions"].append(cls_index)
#         else:
#             start_char = answers["answer_start"][0]
#             end_char = start_char + len(answers["text"][0])

#             token_start_index = 0
#             while sequence_ids[token_start_index] != 1:
#                 token_start_index += 1
#             token_end_index = len(input_ids) - 1
#             while sequence_ids[token_end_index] != 1:
#                 token_end_index -= 1

#             start_position = cls_index
#             end_position = cls_index
#             for idx in range(token_start_index, token_end_index + 1):
#                 start, end = offsets[idx]
#                 if start <= start_char < end:
#                     start_position = idx
#                 if start < end_char <= end:
#                     end_position = idx

#             tokenized_examples["start_positions"].append(start_position)
#             tokenized_examples["end_positions"].append(end_position)

#     return tokenized_examples

# # Tokenize dataset
# tokenized_datasets = dataset.map(
#     prepare_train_features,
#     batched=True,
#     remove_columns=dataset["train"].column_names
# )

In [14]:

# Load base model
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

# Setup LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query", "key", "value", "dense"],
    lora_dropout=0.05,
    bias="lora_only",
    task_type=TaskType.QUESTION_ANS,
    inference_mode=False  # Explicitly set for training

)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Show trainable params
model.print_trainable_parameters()


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,738,690 || all params: 111,548,932 || trainable%: 2.4551


In [15]:
# import evaluate
# import numpy as np

# # Load the SQuAD v2 metric
# metric = evaluate.load("squad_v2")

# def compute_metrics(p):
#     predictions, references = p

#     # The predictions are a tuple containing start and end logits
#     pred_start = np.argmax(predictions[0], axis=1)
#     pred_end = np.argmax(predictions[1], axis=1)

#     # Convert the start and end indices to answers using the tokenizer
#     pred_answers = []
#     for i in range(len(pred_start)):
#         start = int(pred_start[i])  # Ensure start is an integer
#         end = int(pred_end[i])  # Ensure end is an integer
#         answer = tokenizer.decode(predictions[0][i][start:end+1], skip_special_tokens=True)
#         pred_answers.append(answer)

#     # Extract the references' answers
#     ref_answers = [references["answers"]["text"][i] for i in range(len(references["answers"]["text"]))]

#     # Convert to SQuAD format for evaluation
#     result = metric.compute(predictions={"id": references["id"], "prediction_text": pred_answers},
#                             references={"id": references["id"], "answers": ref_answers})

#     return result


In [16]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [17]:
# split_dataset['test'][0]

In [18]:
# from datasets import load_metric
# import numpy as np

# squad_metric = load_metric("squad_v2")

# def compute_metrics(p):
#     predictions, labels = p
#     start_preds = np.argmax(predictions.predictions[0], axis=-1)
#     end_preds = np.argmax(predictions.predictions[1], axis=-1)
#     start_labels = labels["start_positions"]
#     end_labels = labels["end_positions"]
    
#     # You will need to map predictions to text and compute EM/F1 for real accuracy.
#     # Usually needs post-processing step (depends on tokenizer, context, etc.)
    
#     return squad_metric.compute(predictions=start_preds, references=start_labels)

In [19]:
# small_train_dataset = split_dataset["train"].shuffle(seed=42).select(range(50_000))

In [20]:
from transformers import TrainingArguments, Trainer
label_names=["start_positions", "end_positions"]

args = TrainingArguments(
    output_dir="/kaggle/working/qa_model",
    run_name="bert-lora-squad2_V2",
    eval_strategy="steps",
    eval_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    lr_scheduler_type="linear",
    report_to="wandb",
    save_strategy="steps",            # <- Save during steps instead of epoch
    save_steps=1000,                  # <- Save every 1000 steps
    save_total_limit=2,               # <- Keep only last 2 checkpoints to save space
    load_best_model_at_end=True,      # <- Optional: loads best model (by metric)
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    label_names=["start_positions", "end_positions"]  

)

In [21]:
from transformers import default_data_collator
from transformers import EarlyStoppingCallback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop if no improvement after 3 evals
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    callbacks=[early_stopping]
    
)

trainer.train()

/tmp/ipykernel_19/2457908637.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: Currently logged in as: mohamedshakshak455 (mohamedshakshak455-bfcai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250502_125911-o6i5953c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bert-lora-squad2_V2
wandb: ⭐️ View project at https://wandb.ai/mohamedshakshak455-bfcai/huggingface
wandb: 🚀 View run at https://wandb.ai/mohamedshakshak455-bfcai/huggingface/runs/o6i5953c


Step,Training Loss,Validation Loss
1000,2.960300,2.377988
2000,2.276700,1.838478
3000,2.051500,1.718082
4000,1.884600,1.716755
5000,1.760600,1.606521
6000,1.746300,1.576377
7000,1.709300,1.550215
8000,1.635300,1.517322
9000,1.602300,1.434963
10000,1.587500,1.447665


TrainOutput(global_step=19000, training_loss=1.7971840450889185, metrics={'train_runtime': 13861.9544, 'train_samples_per_second': 34.223, 'train_steps_per_second': 2.139, 'total_flos': 6.143255406364262e+16, 'train_loss': 1.7971840450889185, 'epoch': 2.5630648860110616})

In [22]:
trainer.save_model("/kaggle/working/bert-lora-squad2_V2")
tokenizer.save_pretrained("/kaggle/working/bert-lora-squad2_V2")
model.save_pretrained("/kaggle/working/bert-lora-squad2_V2_model")

In [23]:
from transformers import pipeline
from peft import PeftModel, PeftConfig

# Load base model
base_model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, "/kaggle/working/bert-lora-squad2_V2")
model = model.merge_and_unload()  # Merge for faster inference

# Create pipeline with your trained model
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [24]:
import evaluate
import numpy as np
from tqdm import tqdm
metric = evaluate.load("squad_v2")

for example in tqdm(test_data):
    # Get prediction using YOUR trained model
    result = qa_pipeline(
        question=example["question"],
        context=example["context"]
    )
    prediction_text = result["answer"].lower()

    # Format prediction
    formatted_prediction = {
        "id": example["id"],
        "prediction_text": prediction_text,
        "no_answer_probability": 0.0 if prediction_text else 1.0
    }

    # Format reference
    formatted_reference = {
        "id": example["id"],
        "answers": {
            "text": [ans.lower() for ans in example["answers"]["text"]],
            "answer_start": example["answers"]["answer_start"]
        }
    }

    metric.add(prediction=formatted_prediction, reference=formatted_reference)

# Get final metrics
final_metrics = metric.compute()
print(f"Exact Match: {final_metrics['exact']:.2f}")
print(f"F1 Score: {final_metrics['f1']:.2f}")

100%|██████████| 13032/13032 [02:54<00:00, 74.55it/s]


Exact Match: 37.16
F1 Score: 45.95


In [25]:
example = {
    "question": "Who developed the theory of relativity?",
    "context": "Albert Einstein developed the theory of relativity in the early 20th century, which revolutionized physics."
}

result = qa_pipeline(example)
print("Answer:", result["answer"])


Answer: Albert Einstein


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [26]:
examples = [
    {
        "question": "What is the capital of France?",
        "context": "Paris is the capital and most populous city of France."
    },
    {
        "question": "When was the iPhone first released?",
        "context": "The first iPhone was released by Apple Inc. on June 29, 2007."
    }
]

for example in examples:
    answer = qa_pipeline(example)
    print(f"Q: {example['question']}\nA: {answer}\n")


Q: What is the capital of France?
A: {'score': 0.7676910758018494, 'start': 0, 'end': 5, 'answer': 'Paris'}

Q: When was the iPhone first released?
A: {'score': 0.7373287677764893, 'start': 47, 'end': 60, 'answer': 'June 29, 2007'}



In [27]:
# from huggingface_hub import HfApi, HfFolder
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# # Optional: Set repo name (this will be your model's name on HF)
# repo_name = "bert-qa-squad2_V2"  # change as you like

# # Create the repo (if it doesn’t exist yet)
# from huggingface_hub import create_repo
# create_repo(repo_name, private=True)  # private or public

# # Push model
# model.push_to_hub(repo_name)
# tokenizer.push_to_hub(repo_name)
